In [15]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import pyspark
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import col, isnan, when, count
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
spark = pyspark.sql.SparkSession.builder.getOrCreate() #init pyspark

In [18]:
df4 = spark.read.csv('/content/drive/MyDrive/datasets/Parking_Violations_Issued_-_Fiscal_Year_2017.csv', sep=',', inferSchema=True, header=True, multiLine=True)

In [19]:
print('Dataframe shape:', (df4.count(), len(df4.columns)))

Dataframe shape: (10803028, 43)


In [20]:
df = df4.dropDuplicates() #drop duplicates

In [21]:
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show() #null values in columns

+--------------+--------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+--------------+------------+--------------+-------------------+----------------+---------------------------------+------------+-----------+-------------------+-------------------+-----------+------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+------------+------------+--------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+
|Summons Number|Plate ID|Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Vehicle Expiration Date|Violation Location|Violation Precinct|Issuer Precinct|Issuer Code

In [22]:
#drop the columns with over 50% null values
drop_cols = ['Time First Observed', 'Intersecting Street', 'Violation Legal Code', 'Unregistered Vehicle?', 'Meter Number', 'No Standing or Stopping Violation', 'Hydrant Violation', 'Double Parking Violation']
df_new = df.drop(*drop_cols)

In [23]:
dfWoutNull = df_new.na.drop() #drop the null values
print('Dataframe shape:',(dfWoutNull.count(), len(dfWoutNull.columns)))

Dataframe shape: (4190048, 35)


In [24]:
df1m = dfWoutNull.sample(0.3)
dfP = df1m.toPandas() #convert the spark dataframe to pandas
dfP.to_csv('parking2017.csv') #save it to csv file